Ideally, go so far as to prototype it, making functions that hardwire the notebook examples so you can see how it would work. Think about how it could work not as below with astroquery.heasarc.list_sia_services(), for example, but just as astroquery.list_sia_services(). The whole point is the user should care which site it comes from.

If HEASARC takes over astroquery.heasarc, we could use things like:

TAP:  

    astroquery.heasarc.list_sia_services() 

TAP:  

    astroquery.heasarc.surveys_like("Redshift”)

SIA:  
    
    astroquery.heasarc.query_skyview()

SIA:  
    
    astroquery.heasarc.query_image()  # Xamin instead


 notes on Vandana’s collected workshop notebook:


In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import requests, io, astropy
from IPython.display import Image, display

## For handling ordinary astropy Tables
from astropy.table import Table

## For reading FITS files
import astropy.io.fits as apfits

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

General use regtap function for use internally only, i.e. users shouldn't need to see this:

In [5]:
def regtap_get_service( source='' , service_type='' ):
    
    regtap_url='http://vao.stsci.edu/RegTAP/TapService.aspx/sync'
    if 'image' in service_type:
        service_type='simpleimageaccess'
    elif 'cone' in service_type: 
        service_type="conesearch"
    elif 'spectr' in service_type:
        service_type='simplespectralaccess'
    else:
        print("ERROR: don't recognize the requested service type")
        raise
        

    criteria="where a.cap_type='{}'".format(service_type)    
    if source is not '':
        criteria=criteria + " and a.ivoid like 'ivo://%{}%'".format(source)

    # Warning:  'query' is bizarrely case-sensitive:
    tap_params = {
        "request":"doQuery",
        "lang":"ADQL",
        "query":"""
            select b.short_name,b.res_description, c.access_url 
            from rr.capability a natural join rr.resource b natural join rr.interface c
            {} 
            order by short_name;
        """.format(criteria)
        }

    r = requests.post(regtap_url, data=tap_params)
    ## The astropy.table  module will read this VO Table formatted result into an astropy Table:
    return Table.read(io.BytesIO(r.content))
    


3 NED example cone search:  

Instead of searching NED ‘manually’, a generic cone search that you can give a list of ras, decs, and radii (or just one obviously) and optionally specify that you want ‘ned’ results or some other IVOID substring. If you ask for known things like ‘ned’, it has hard-wired (?) base URL and query path. If not, it queries the RegTAP to find out what cone searches are available that match a given ivoid_string.  So the user would call:

    cone_results = astroquery.cone_search(ras, decs, radii, [source={‘heasarc’, ‘irsa’,’stsci’,’ned’,’ivoid_string'}] ) 

where ras, decs, and radii can be floats or arrays of floats, get back table of NED objects (or all?), including the column ACREF  that the user doesn’t want to have to know about. Or one function that calls a cone search for a single position and returns the info for all objects found in a list of tables:

    ned_info = astroquery.get_ned_info( ra, dec, radii )

calls the cone search and passes the ACREF for each match to NED again. 

Questions: HEASARC cone search doesn’t return an ACREF. Why not? The columns returned aren’t part of the standard? Ned returns ObjectName, NumberPhotometricPoints, and ACREF. HEASARC’s chanmaster returns obsid, status, name, ra, dec, time, detector, … So these things aren’t in any way standard, only the way you ask for ra, dec, and radius? In which case, the get_ned_info() couldn’t be generalized. 


In [ ]:

def cone_search( ras, decs, radii, source='' ):
    # Define a list of URLs to query. If source given, use 
    return Table()

4 TAP:  

    sia_services = astroquery.list_sia_services( [source={‘heasarc’, ‘irsa’,’stsci’,’ned’,’ivoid_string'}] , [name_like=‘allwise’], [description_like=‘whatever’] ) 

This one can easily be generalized so you can get images from any service (or a chosen one) that have a short name substring, or description substring. (Or perhaps just string_like=‘whatever’ and it searches both short name as well as description?) It returns a table of information, including the ‘access_url’ that you can then plug into another generic function




In [ ]:
def list_sia_services( source='', name_like=''):
    return Table()

5 SIA

    sia_results = astroquery.get_image( access_url=sia_services[20][‘access_url’], pos=‘185.47873,4.47365', size=‘0’]

or perhaps

    sia_results = astroquery.get_image_info( access_url=sia_services[:][‘access_url’], pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’]

to get a table list of all images in a list of services that contain that point. But combining the columns from the different services is again annoying. Need to use a function that combines them using the UCD? Eg., HEASARC uses “sia_url”, and this is the UCD “VOX:Image_AccessReference”, so something inside would have transparently translate from each different archive’s column names to something standardized. (The UCD itself? Or something we define, e.g., “image_url”?)



In [ ]:
def get_image_info( access_url, pos, size, naxis=''):
    return Table()



7  Then get it with a trivial wrapper of urllib.request ? For HEASARC SkyView, can get FITS or JPEG. Others? Generalized?

    astroquery.download_image( sia_results[0][‘image_url’], [filename,] format={‘fits’,’jpeg’} )

With the option to rename it. Or better to read it in immediately? 

    fits_image = astroquery.get_image( sia_results[0][‘image_url’] )
    plt.imshow( fits_image )

and fits_image is the data array from the image extension itself. Or should we get the HUD list itself?

    fits_image =  astroquery.get_image( sia_results[0][‘image_url’] )
    plt.imshow( fits_image[0].data )



8

This is complex but can use the above. 

    galex_sia_services = astroquery.list_sia_services( name_like=“galex”) 
    2mass_sia_services = astroquery.list_sia_services( name_like=“2mass”) 
    allwise_sia_services = astroquery.list_sia_services( name_like=“allwise”) 
    
    query_urls=galex_sia_services[:][’access_url’]
    query_urls.extend( 2mass_sia_services[:][’access_url’])
    query_urls.extend( allwise_sia_services[:][’access_url’])
    
    for url in query_urls:
    



    